In [2]:
import pandas as pd
import numpy as np
import ipaddress
import dns.resolver
import dns.reversename
import pygeoip
import matplotlib.pyplot as plt 

anomalous_datafile='test5.parquet'
normal_datafile='data5.parquet'

### IP geolocalization
gi=pygeoip.GeoIP('./GeoIP.dat')
gi2=pygeoip.GeoIP('./GeoIPASNum.dat')
addr='193.136.73.21'
cc=gi.country_code_by_addr(addr)
org=gi2.org_by_addr(addr)
print(cc,org)

### DNS resolution
addr=dns.resolver.resolve("www.ua.pt", 'A')
for a in addr:
    print(a)
    
### Reverse DNS resolution    
name=dns.reversename.from_address("193.136.172.20")
addr=dns.resolver.resolve(name, 'PTR')
for a in addr:
    print(a)

### Read parquet data files
anamalous_data=pd.read_parquet(anomalous_datafile)
normal_data=pd.read_parquet(normal_datafile)

# Volume of traffic per Upload
# Average number of uploaded bytes per flow

# Sum all the up_bytes in the column up_bytes
sumUpBytes=anamalous_data['up_bytes'].sum()
print(sumUpBytes)

# Calculate the average of the up_bytes in the column up_bytes
avgUpBytes=anamalous_data['up_bytes'].mean()
print(avgUpBytes)

# Calculate the median of the up_bytes in the column up_bytes
medianUpBytes=anamalous_data['up_bytes'].median()
print(medianUpBytes)

# Calculate the standard deviation of the up_bytes in the column up_bytes
stdUpBytes=anamalous_data['up_bytes'].std()
print(stdUpBytes)

PT AS1930 Fundacao para a Ciencia e a Tecnologia, I.P.
193.136.173.58
dns.ua.pt.
16833951313
16378.306817341558
8664.0
1172915.7149008764


In [4]:
### Read parquet data files
anomalous_data=pd.read_parquet(anomalous_datafile)
normal_data=pd.read_parquet(normal_datafile)

normal = normal_data[['up_bytes','down_bytes']].describe()
anomal = anamalous_data[['up_bytes','down_bytes']].describe()

In [4]:
# Para o método do desvio padrão
mean_up = normal_data['up_bytes'].mean()
std_up = normal_data['up_bytes'].std()
threshold_up = mean_up + 3*std_up

mean_down = normal_data['down_bytes'].mean()
std_down = normal_data['down_bytes'].std()
threshold_down = mean_down + 3*std_down

# Para o método IQR
Q1_up = normal_data['up_bytes'].quantile(0.25)
Q3_up = normal_data['up_bytes'].quantile(0.75)
IQR_up = Q3_up - Q1_up
lower_bound_up = Q1_up - 1.5*IQR_up
upper_bound_up = Q3_up + 1.5*IQR_up

Q1_down = normal_data['down_bytes'].quantile(0.25)
Q3_down = normal_data['down_bytes'].quantile(0.75)
IQR_down = Q3_down - Q1_down
lower_bound_down = Q1_down - 1.5*IQR_down
upper_bound_down = Q3_down + 1.5*IQR_down


In [11]:
# Carregando o conjunto de dados de teste
test_data = pd.read_parquet('test5.parquet')

# Identificando anomalias
anomalies_up = test_data[(test_data['up_bytes'] > threshold_up) | (test_data['up_bytes'] < lower_bound_up) | (test_data['up_bytes'] > upper_bound_up)]
# in the table anomalies_up add the countries and organizations from source IP and destination IP
anomalies_up['dst_country'] = anomalies_up['dst_ip'].apply(lambda x: gi.country_code_by_addr(x))
anomalies_up['dst_org'] = anomalies_up['dst_ip'].apply(lambda x: gi2.org_by_addr(x))

anomalies_down = test_data[(test_data['down_bytes'] > threshold_down) | (test_data['down_bytes'] < lower_bound_down) | (test_data['down_bytes'] > upper_bound_down)]
# in the table anomalies_down add the countries and organizations from source IP and destination IP
anomalies_down['dst_country'] = anomalies_down['dst_ip'].apply(lambda x: gi.country_code_by_addr(x))
anomalies_down['dst_org'] = anomalies_down['dst_ip'].apply(lambda x: gi2.org_by_addr(x))



/tmp/ipykernel_129368/719860936.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomalies_up['dst_country'] = anomalies_up['dst_ip'].apply(lambda x: gi.country_code_by_addr(x))
/tmp/ipykernel_129368/719860936.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomalies_up['dst_org'] = anomalies_up['dst_ip'].apply(lambda x: gi2.org_by_addr(x))
/tmp/ipykernel_129368/719860936.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [9]:
# from all data, get the organization and country of the source IP and destination IP
test_data['src_country'] = test_data['src_ip'].apply(lambda x: gi.country_code_by_addr(x))
test_data['src_org'] = test_data['src_ip'].apply(lambda x: gi2.org_by_addr(x))
test_data['dst_country'] = test_data['dst_ip'].apply(lambda x: gi.country_code_by_addr(x))
test_data['dst_org'] = test_data['dst_ip'].apply(lambda x: gi2.org_by_addr(x))


KeyboardInterrupt: 